# Complex Example

This notebook demonstrates advanced features of **flixopt** including:

- **Investment optimization** with piecewise cost functions
- **Status parameters** for startup costs and operating constraints
- **Piecewise conversion** for non-linear component behavior
- **Multiple effects** (costs, CO2, primary energy)
- **Saving and loading** results

This example builds a more realistic energy system model with operational constraints.

## Setup

In [ ]:
import numpy as np
import pandas as pd

import flixopt as fx

fx.CONFIG.notebook()

## Configuration Options

These options allow experimenting with different model configurations:

In [ ]:
# Experiment options
check_penalty = False  # Set True to test imbalance penalties
imbalance_penalty = 1e5  # High penalty for unmet demand
use_chp_with_piecewise_conversion = True  # Use piecewise CHP model

## Define Input Data

Create demand profiles and electricity prices:

In [ ]:
# Demand profiles
electricity_demand = np.array([70, 80, 90, 90, 90, 90, 90, 90, 90])
heat_demand = (
    np.array([30, 0, 90, 110, 2000, 20, 20, 20, 20])
    if check_penalty
    else np.array([30, 0, 90, 110, 110, 20, 20, 20, 20])
)
electricity_price = np.array([40, 40, 40, 40, 40, 40, 40, 40, 40])

# Create time index
timesteps = pd.date_range('2020-01-01', periods=len(heat_demand), freq='h')

print(f'Heat demand range: {heat_demand.min()} - {heat_demand.max()} kW')

## Create FlowSystem with Buses

We include an `imbalance_penalty` on each bus to handle cases where supply cannot meet demand:

In [ ]:
flow_system = fx.FlowSystem(timesteps)

flow_system.add_elements(
    fx.Bus('Strom', carrier='electricity', imbalance_penalty_per_flow_hour=imbalance_penalty),
    fx.Bus('Fernwärme', carrier='heat', imbalance_penalty_per_flow_hour=imbalance_penalty),
    fx.Bus('Gas', carrier='gas', imbalance_penalty_per_flow_hour=imbalance_penalty),
)

## Define Effects

Three effects track different aspects of the system:

In [ ]:
# Costs - objective function with CO2 price linkage
Costs = fx.Effect(
    'costs',
    '€',
    'Kosten',
    is_standard=True,
    is_objective=True,
    share_from_temporal={'CO2': 0.2},  # 0.2 €/kg CO2
)

# CO2 emissions
CO2 = fx.Effect('CO2', 'kg', 'CO2_e-Emissionen')

# Primary energy with total limit
PE = fx.Effect('PE', 'kWh_PE', 'Primärenergie', maximum_total=3.5e3)

## Component 1: Gas Boiler with Investment

This boiler includes:
- **Investment parameters** with fixed size
- **Status parameters** for operating constraints (min/max uptime, startup costs)
- **Load constraints** (minimum and maximum part load)

In [ ]:
Gaskessel = fx.linear_converters.Boiler(
    'Kessel',
    thermal_efficiency=0.5,
    status_parameters=fx.StatusParameters(
        effects_per_active_hour={Costs.label: 0, CO2.label: 1000}  # CO2 when running
    ),
    thermal_flow=fx.Flow(
        label='Q_th',
        bus='Fernwärme',
        size=fx.InvestParameters(
            effects_of_investment=1000,  # Fixed investment cost
            fixed_size=50,  # Fixed capacity
            mandatory=True,  # Must be built
            effects_of_investment_per_size={Costs.label: 10, PE.label: 2},  # Per-kW costs
        ),
        load_factor_max=1.0,
        load_factor_min=0.1,
        relative_minimum=5 / 50,
        relative_maximum=1,
        previous_flow_rate=50,  # Initial state
        flow_hours_max=1e6,  # Total energy limit
        status_parameters=fx.StatusParameters(
            active_hours_min=0,
            active_hours_max=1000,
            max_uptime=10,
            min_uptime=np.array([1, 1, 1, 1, 1, 2, 2, 2, 2]),  # Variable min uptime
            max_downtime=10,
            effects_per_startup={Costs.label: 0.01},  # Startup cost
            startup_limit=1000,
        ),
    ),
    fuel_flow=fx.Flow(label='Q_fu', bus='Gas', size=200),
)

## Component 2: Standard CHP

A simple CHP with constant efficiency ratios:

In [ ]:
bhkw = fx.linear_converters.CHP(
    'BHKW2',
    thermal_efficiency=0.5,
    electrical_efficiency=0.4,
    status_parameters=fx.StatusParameters(effects_per_startup={Costs.label: 0.01}),
    electrical_flow=fx.Flow('P_el', bus='Strom', size=60, relative_minimum=5 / 60),
    thermal_flow=fx.Flow('Q_th', bus='Fernwärme', size=1e3),
    fuel_flow=fx.Flow('Q_fu', bus='Gas', size=1e3, previous_flow_rate=20),
)

## Component 3: Piecewise CHP

For more realistic modeling, efficiency can vary with load. The **piecewise conversion** defines segments where conversion ratios change:

```
P_el: [5-30] → [40-60]  (two operating segments)
Q_th: [6-35] → [45-100]
Q_fu: [12-70] → [90-200]
```

In [ ]:
# Define flows for piecewise CHP
P_el = fx.Flow('P_el', bus='Strom', size=60, previous_flow_rate=20)
Q_th = fx.Flow('Q_th', bus='Fernwärme')
Q_fu = fx.Flow('Q_fu', bus='Gas')

# Define piecewise conversion with two operating segments
piecewise_conversion = fx.PiecewiseConversion(
    {
        P_el.label: fx.Piecewise([fx.Piece(5, 30), fx.Piece(40, 60)]),
        Q_th.label: fx.Piecewise([fx.Piece(6, 35), fx.Piece(45, 100)]),
        Q_fu.label: fx.Piecewise([fx.Piece(12, 70), fx.Piece(90, 200)]),
    }
)

bhkw_2 = fx.LinearConverter(
    'BHKW2',
    inputs=[Q_fu],
    outputs=[P_el, Q_th],
    piecewise_conversion=piecewise_conversion,
    status_parameters=fx.StatusParameters(effects_per_startup={Costs.label: 0.01}),
)

## Component 4: Storage with Piecewise Investment

Investment costs often have economies of scale. Here we model this with piecewise investment effects:

- Capacity range: 5-100 kWh
- Costs: non-linear (cheaper per kWh for larger storage)

In [ ]:
# Piecewise investment effects (economies of scale)
segmented_investment_effects = fx.PiecewiseEffects(
    piecewise_origin=fx.Piecewise([fx.Piece(5, 25), fx.Piece(25, 100)]),
    piecewise_shares={
        Costs.label: fx.Piecewise([fx.Piece(50, 250), fx.Piece(250, 800)]),
        PE.label: fx.Piecewise([fx.Piece(5, 25), fx.Piece(25, 100)]),
    },
)

speicher = fx.Storage(
    'Speicher',
    charging=fx.Flow('Q_th_load', bus='Fernwärme', size=1e4),
    discharging=fx.Flow('Q_th_unload', bus='Fernwärme', size=1e4),
    capacity_in_flow_hours=fx.InvestParameters(
        piecewise_effects_of_investment=segmented_investment_effects,
        mandatory=True,
        minimum_size=0,
        maximum_size=1000,
    ),
    initial_charge_state=0,
    maximal_final_charge_state=10,
    eta_charge=0.9,
    eta_discharge=1,
    relative_loss_per_hour=0.08,
    prevent_simultaneous_charge_and_discharge=True,
)

## Define Sinks and Sources

In [ ]:
# Heat demand
Waermelast = fx.Sink(
    'Wärmelast',
    inputs=[
        fx.Flow(
            'Q_th_Last',
            bus='Fernwärme',
            size=1,
            fixed_relative_profile=heat_demand,
        )
    ],
)

# Gas supply
Gasbezug = fx.Source(
    'Gastarif',
    outputs=[
        fx.Flow(
            'Q_Gas',
            bus='Gas',
            size=1000,
            effects_per_flow_hour={Costs.label: 0.04, CO2.label: 0.3},
        )
    ],
)

# Electricity feed-in
Stromverkauf = fx.Sink(
    'Einspeisung',
    inputs=[
        fx.Flow(
            'P_el',
            bus='Strom',
            effects_per_flow_hour=-1 * electricity_price,  # Revenue
        )
    ],
)

## Build and Visualize FlowSystem

In [ ]:
# Add components
flow_system.add_elements(Costs, CO2, PE, Gaskessel, Waermelast, Gasbezug, Stromverkauf, speicher)

# Choose CHP type based on configuration
if use_chp_with_piecewise_conversion:
    flow_system.add_elements(bhkw_2)
else:
    flow_system.add_elements(bhkw)

# Print system overview
print(flow_system)

## Run Optimization

In [ ]:
flow_system.optimize(fx.solvers.HighsSolver(mip_gap=0.01, time_limit_seconds=60))

## Save Results

Results can be saved to NetCDF format for later analysis:

In [ ]:
# Save to file (uncomment to save)
# flow_system.to_netcdf('results/complex_example.nc')

## Analyze Results

In [ ]:
flow_system.statistics.plot.heatmap('BHKW2(Q_th)')

In [ ]:
flow_system.statistics.plot.balance('BHKW2')

In [ ]:
flow_system.statistics.plot.heatmap('Speicher')

In [ ]:
flow_system.statistics.plot.balance('Fernwärme')

## Summary

This example demonstrated advanced flixopt features:

- **InvestParameters** for component sizing optimization
- **StatusParameters** for operational constraints (min uptime, startup costs)
- **PiecewiseConversion** for non-linear component behavior
- **PiecewiseEffects** for economies of scale in investment
- **Multiple effects** with constraints and cross-linkage
- **Result persistence** via NetCDF

For even more advanced optimization modes, see the optimization modes notebook!